In [3]:
import pandas as pd
import re
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.2MB 6.0MB/s 
     |████████████████████████████████| 870kB 33.1MB/s 
     |████████████████████████████████| 3.3MB 42.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=7b6a57eef5b21628d213b2b7406f42574be63e886840b6e457bceaf627e20a47
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

In [47]:
df = pd.read_csv("/content/TM Bot v1.0 - utterances.csv")
df = df[["Utterance", "Intent"]]

In [48]:
sentences = df.dropna().values

In [54]:
sentences[0, 0]

'i did not get any email confirmation and cannot log in to my account with <email>'

In [53]:
def paraphrases(sentence):
  text =  "paraphrase: " + sentence + " </s>"
  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.98,
      early_stopping=True,
      num_return_sequences=4
  )


  final_outputs =[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)

  return final_outputs

In [61]:
data = []
for utterance, intent in sentences:
  temp = paraphrases(utterance)
  for t in temp:
    data.append((t, intent, utterance))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


In [62]:
finalData = pd.DataFrame.from_records(data, columns =['new_utt', 'intent', 'original_utt'])

In [63]:
finalData.sample(30)

,new_utt,intent,original_utt
653,What is a physical address where you will be p...,tour_inquiry_pick_up,Is there a physical address where we will be p...
377,Should I send an email to support about reserv...,reservation_update_request,I sent an email to support about reservation <...
41,I need keyword> tickets for date>. I need keyw...,book_tour,I need <keyword> tickets for <date>
138,What should I do with time?,recommend_activity,What should I do <time>?
547,Does a keyword in a location have this key?,tour_available,Do you have this <keyword> in <location>?
345,Which one confirms reservation when we pay?,reservation_regulations,Is this a confirmed reservation if we pay?
324,I need bill copy for purchases made by my boss...,reservation_inquiry,I need the bill copy of a purchase made by my ...
622,What is happened if I don't get reservation on...,tour_inquiry,What happens if I don't get the reservation on...
261,How can you show me a less crowded location if...,recommend_poi,Show me less crowded locations
133,What can I do in location time?,recommend_activity,What can I do in <location> <time>?


In [60]:
import gc

gc.collect()

3568

In [64]:
finalData.to_csv("SemanticSimilar.csv")